In [ ]:
from djitellopy import Tello
import cv2
import pygame
import numpy as np
import time
from tflite_runtime.interpreter import Interpreter
import threading

##### TODO: UPDATE THE PARAMETERS IN THIS SECTION #####

label_path = 'dict.txt'
model_path = 'model.tflite'
confidence_threshold = 0.4

##### END OF SECTION #####

# Speed of the drone
S = 60  # Change to 60 - 80 for real comp
# Frames per second of the pygame window display
FPS = 120
Hori_Increment = 200  # Change to 250 for real comp
Vert_Increment = 80  # Change to 80 - 90 for real comp
# Settings for frame size
FRAME_WIDTH = 960
FRAME_HEIGHT = 660

class FrontEnd(object):
    """
    Maintains the Tello display and moves it through the keyboard keys.
    Press escape key to quit.
    Press enter key to take a snapshot.
    The controls are:
    - T: Takeoff
    - L: Land
    - Arrow keys: Forward, backward, left, right
    - A and D: Counter clockwise, clockwise rotations (yaw)
    - W and S: Up, down
    """
    
    def __init__(self):
        # Init pygame
        pygame.init()

        # Create pygame window 
        pygame.display.set_caption("Tello video stream")
        # Set width and height
        self.screen = pygame.display.set_mode([FRAME_WIDTH, FRAME_HEIGHT])

        # Init Tello object that interacts with the Tello drone
        self.tello = Tello()

        # Drone velocities between -100~100
        self.for_back_velocity = 0
        self.left_right_velocity = 0
        self.up_down_velocity = 0
        self.yaw_velocity = 0
        self.speed = 10
        self.Current_Gate = 1

        # Initialize snapshot time 
        self.last_snapshot = "N/A"
        self.text_color = (0, 0, 255)
        
        self.send_rc_control = False

        # Import labels 
        with open(label_path, "r") as f:
            self.labels = [line.strip() for line in f.readlines()]

        # Initialize TFLite model and allocate tensors
        self.interpreter = Interpreter(model_path=model_path)
        self.interpreter.allocate_tensors()
        self.input_details = self.interpreter.get_input_details()
        self.output_details = self.interpreter.get_output_details()

        # Create update timer
        pygame.time.set_timer(pygame.USEREVENT + 1, 1000 // FPS)

        self.frame_read = None
        self.new_image = None
        self.running = True

    def run(self):
        self.tello.connect()
        self.tello.set_speed(self.speed)

        # In case streaming is on. This happens when we quit this program without the escape key.
        self.tello.streamon()
        self.tello.set_video_fps(Tello.FPS_30)
        self.Scores = 0

        self.frame_read = self.tello.get_frame_read()

        # Start the image processing thread
        threading.Thread(target=self.update, daemon=True).start()

        while self.running:
            for event in pygame.event.get():
                if event.type == pygame.USEREVENT + 1:
                    if self.new_image is not None:
                        frame = pygame.surfarray.make_surface(self.new_image)
                        self.screen.blit(frame, (0, 0))
                        pygame.display.update()
                elif event.type == pygame.QUIT or (event.type == pygame.KEYDOWN and event.key == pygame.K_ESCAPE):
                    self.running = False

        self.tello.end()

    def update(self):
        while self.running:
            time.sleep(1 / FPS)
            
            original_shape = np.shape(self.frame_read.frame)
            input_shape = self.input_details[0]['shape']
            new_image = cv2.resize(self.frame_read.frame, (input_shape[1], input_shape[2]))

            self.interpreter.set_tensor(self.input_details[0]['index'], [new_image])
            self.interpreter.invoke()

            boxes = self.interpreter.get_tensor(self.output_details[0]['index']).squeeze()
            classes = self.interpreter.get_tensor(self.output_details[1]['index']).squeeze()
            scores = self.interpreter.get_tensor(self.output_details[2]['index']).squeeze()

            for i in range(len(scores)):
                if scores[i] > confidence_threshold:
                    self.Scores = scores[i]
                    print(f"Score: {self.Scores}, Class index: {int(classes[i])}")
                    try:
                        print(f"Label: {self.labels[int(classes[i])]}")
                    except IndexError:
                        print(f"IndexError: Class index {int(classes[i])} is out of range for labels list of length {len(self.labels)}")
                        continue

                    unnormed_coords = boxes[i] * input_shape[1]
                    start_point = (int(unnormed_coords[1]), int(unnormed_coords[0]))
                    end_point = (int(unnormed_coords[3]), int(unnormed_coords[2]))
                    cv2.rectangle(new_image, start_point, end_point, color=(0, 255, 0), thickness=2)
                    img_text = f"{self.labels[int(classes[i])]}: {scores[i]:.3f}"
                    cv2.putText(new_image, img_text, (int(unnormed_coords[1]), int(unnormed_coords[0]) + 15), cv2.FONT_HERSHEY_SIMPLEX, 0.4, (0, 255, 0), 1)
                else:
                    break

            new_image = cv2.resize(new_image, (original_shape[1], original_shape[0]))
            text = "Battery: {}%".format(self.tello.get_battery())
            cv2.putText(new_image, text, (5, FRAME_HEIGHT - 5), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)

            new_image = np.rot90(new_image)
            new_image = np.flipud(new_image)
            self.new_image = new_image

def main():
    frontend = FrontEnd()
    frontend.run()

if __name__ == '__main__':
    main()


pygame 2.6.0 (SDL 2.28.4, Python 3.10.12)
Hello from the pygame community. https://www.pygame.org/contribute.html


[INFO] tello.py - 129 - Tello instance was initialized. Host: '192.168.10.1'. Port: '8889'.
INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
[INFO] tello.py - 438 - Send command: 'command'
[INFO] tello.py - 462 - Response command: 'ok'
[INFO] tello.py - 438 - Send command: 'speed 10'
[INFO] tello.py - 462 - Response speed 10: 'ok'
[INFO] tello.py - 438 - Send command: 'streamon'
[INFO] tello.py - 462 - Response streamon: 'ok'
[INFO] tello.py - 438 - Send command: 'setfps high'
[INFO] tello.py - 462 - Response setfps high: 'ok'


Score: 0.57421875, Class index: 0
Label: Square
Score: 0.57421875, Class index: 0
Label: Square
Score: 0.42578125, Class index: 0
Label: Square
Score: 0.42578125, Class index: 0
Label: Square
Score: 0.625, Class index: 0
Label: Square
Score: 0.625, Class index: 0
Label: Square
Score: 0.44921875, Class index: 0
Label: Square
Score: 0.7890625, Class index: 1
Label: Destination
Score: 0.8359375, Class index: 1
Label: Destination
Score: 0.8359375, Class index: 1
Label: Destination
Score: 0.859375, Class index: 1
Label: Destination
Score: 0.8046875, Class index: 1
Label: Destination
Score: 0.8359375, Class index: 1
Label: Destination
Score: 0.8828125, Class index: 1
Label: Destination
Score: 0.90234375, Class index: 1
Label: Destination
Score: 0.8359375, Class index: 1
Label: Destination
Score: 0.8203125, Class index: 1
Label: Destination
Score: 0.8203125, Class index: 1
Label: Destination
Score: 0.8203125, Class index: 1
Label: Destination
Score: 0.8203125, Class index: 1
Label: Destinatio

Score: 0.625, Class index: 0
Label: Square
Score: 0.5234375, Class index: 0
Label: Square
Score: 0.57421875, Class index: 0
Label: Square
Score: 0.55078125, Class index: 0
Label: Square
Score: 0.6015625, Class index: 0
Label: Square
Score: 0.55078125, Class index: 0
Label: Square
Score: 0.5, Class index: 0
Label: Square
Score: 0.6484375, Class index: 0
Label: Square
Score: 0.76953125, Class index: 0
Label: Square
Score: 0.44921875, Class index: 0
Label: Square
Score: 0.6484375, Class index: 0
Label: Square
